In [1]:
'''
Mount Google Drive, copy data to runtime, and unzip folders

Make sure to put a link to "EC 523 Project" in your main google drive!
'''

from google.colab import drive
drive.mount('/content/drive')

! cp /content/drive/MyDrive/'Deep Learning Proj'/train.zip /content
! cp /content/drive/MyDrive/'Deep Learning Proj'/test.zip /content
! cp /content/drive/MyDrive/'Deep Learning Proj'/val.zip /content
! cp /content/drive/'My Drive'/'Deep Learning Proj'/math.txt /content

# from path will differ depending on where you saved the zip file in Google Drive
! unzip -DD -q  /content/train.zip -d  /content/
! unzip -DD -q  /content/test.zip -d  /content/
! unzip -DD -q  /content/val.zip -d  /content/



Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
replace /content/__MACOSX/._train? [y]es, [n]o, [A]ll, [N]one, [r]ename: N
replace /content/__MACOSX/._test? [y]es, [n]o, [A]ll, [N]one, [r]ename: N
replace /content/__MACOSX/._val? [y]es, [n]o, [A]ll, [N]one, [r]ename: N


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
'''
Length of datasets
'''
num_train_str = !ls train | wc -l
num_test_str = !ls test | wc -l
num_val_str = !ls val | wc -l
num_train = int(num_train_str[0])
num_test = int(num_test_str[0])
num_val = int(num_val_str[0])

print(f'Number of train images: {num_train}\nNumber of test images: {num_test}\nNumber of validation images: {num_val}\nTotal images: {num_train+num_test+num_val}')

Number of train images: 158480
Number of test images: 30637
Number of validation images: 6765
Total images: 195882


In [3]:
train_root = "/content/train/"
test_root = "/content/test/"
val_root = "/content/val/"
#label_file = "/content/math.txt"
label_file = "/content/char_labels.txt"
freq_vocabs = "/content/Corpus.txt"

Creating dataset class for images and labels

In [4]:
import os
import cv2
import torch.utils.data
from PIL import Image
import matplotlib.pyplot as plt
import numpy as np


class LatexDataset(torch.utils.data.Dataset):
  def __init__(self, transform=None, dataroot=val_root, max_seq_length = 700): # can change dataroot to be either train_root, test_root, val_root
        '''Initialize the dataset.'''
        self.transform = transform
        self.dataroot = dataroot
        self.labels_txt = label_file
        self.max_seq_length = max_seq_length
        self._parse()

  def _parse(self):
        '''
        Parse the math.txt file.
        Populates the following private variables:
        - self.im_paths: A list of strings storing the associated image paths
        - self.labels: A list of strings, where each string is the latex code for an image
        '''
        def getImPath(idx):
            # Find image in either train, test, or validation folder
            imname = str(idx - 1).zfill(7) + '.png'
            if os.path.exists(f'{self.dataroot}{imname}'):
              impath = f'{self.dataroot}{imname}'
            else:
              return None

            try:
                Image.open(impath).verify()
            except Exception as e:
                # Some images can't be opened
                # print(f"Image at path {impath} is corrupted. Error: {e}")
                return None

            return impath

        self.im_paths = []
        self.labels = []

        with open(self.labels_txt) as f:
            for idx, line in enumerate(f):
                impath = getImPath(idx+1)

                if impath is not None:
                    labels = line.strip('\n')
                    # print("Label length = ", len(labels))
                    if len(labels) < self.max_seq_length-1:
                      # print("PASSED")
                      self.im_paths.append(impath)            # Image name
                      self.labels.append(labels)    # String of latex code


  def __len__(self):
        '''Return length of the dataset.'''
        assert len(self.labels) == len(self.im_paths)
        return len(self.labels)

  def __getitem__(self, index):
        '''
        Return the (image, attributes) tuple.
        This function gets called when you index the dataset.
        '''
        def img_load(index):
            # print(self.im_paths[0:10])
            imraw = Image.open(self.im_paths[index])
            imgray = imraw.convert('L')                         # Convert image to greyscale
            # print("imgray=")
            # print(list(imgray.getdata()))
            imthresh = imgray.point(lambda p: p > 240 and 255)  # Threshold image to remove background (white)
            # img_array = np.array(imthresh)
            # plt.imshow(img_array)
            # plt.axis('off')  # Turn off axis labels
            # plt.show()
            # print("imthresh=")
            # print(imthresh.getdata())
            if self.transform is not None:
              im = self.transform(imthresh)
            else:
              im = imthresh
            return im

        target = self.labels[index]
        # print("target = " ,target)
        return img_load(index), target

Creating dictionary for vocab and tokens

In [5]:
from collections import Counter

def create_dictionary(file_path, n, output_file_path):
    word_count = Counter()

    with open(file_path, 'r') as file:
        # Read the file and count word occurrences
        for line in file:
            words = line.split()
            word_count.update(words)

    # Get the n most common words as a dictionary
    most_common_words = dict(word_count.most_common(n))

    # Save the dictionary to a text file
    ID = 0
    with open(output_file_path, 'w') as output_file:
        for word, count in most_common_words.items():
          if (word != ',') and (word != '\\,'):
            output_file.write(f"{word},{ID} \n")
            ID += 1
        output_file.write(f"{'PAD'},{ID} \n")
        ID += 1
        output_file.write(f"{'UNK'},{ID} \n")

# Example usage
n_most_common = 200  # Change this value to get different numbers of most common words
!touch /content/output.txt
output_file_path = "/content/output.txt" # Replace 'output.txt' with the desired output file path

create_dictionary(label_file, n_most_common, output_file_path)
print(f"Data saved to {output_file_path}")

Data saved to /content/output.txt


In [6]:
'''
Dictionary block: converts a LaTeX string to a dictionary of latex tokens, where
each unique token has its own entry and integer value assigned to it

'''
class LatexDict():
    def __init__(self, max_seq_length=256):
        self.labels_txt = label_file
        self.max_seq_length = max_seq_length
        self.latex_dict = {'<UKN>':0, '<PAD>':1} # Initialize with token for unknown and for padding
        self.latex_dict_inverse = {0:'<UKN>', 1:'<PAD>'} # Initialize inverse dict for quicker reverse lookups
        self.create_dict()

    def create_dict(self):
        # Go through entire label file and populate dictionary
        with open(self.labels_txt) as f:
            for line in f:
                tokens = line.split()
                for token in tokens:
                    if token not in self.latex_dict:
                        # Assign a new ID for the unseen token
                        new_id = len(self.latex_dict)
                        self.latex_dict[token] = new_id
                        self.latex_dict_inverse[new_id] = token

    def map_tokens(self, tex_str_list, batch_size):
        ids_tensor = torch.full((batch_size, self.max_seq_length), self.latex_dict['<PAD>'], dtype=torch.float32)

        for row, tex_str in enumerate(tex_str_list):
            tex_str = r'{ ' + tex_str + ' }'
            tokens = tex_str.split()
            for col, token in enumerate(tokens):
                ids_tensor[row, col] = self.latex_dict[token]

        return ids_tensor

    def tokens_to_tex(self, token_vec):
        tex_str = ' '
        for token_id in token_vec.tolist():
            if token_id in self.latex_dict_inverse:
                if self.latex_dict_inverse[token_id] != '<PAD>' and self.latex_dict_inverse[token_id] != '<UKN>':
                    tex_str += self.latex_dict_inverse[token_id] + ' '

        return tex_str

    def __dict__(self):
        return self.latex_dict

    def __len__(self):
        return len(self.latex_dict)

latex_dict = LatexDict()

In [7]:
'''
Dictionary block: converts a LaTeX string to a dictionary of latex tokens, where
each unique token has its own entry and integer value assigned to it

'''
class LatexDictModified():
    def __init__(self, max_seq_length=256):
        self.labels_txt = label_file
        self.max_seq_length = max_seq_length
        self.latex_dict = {} # Initialize with token for unknown and for padding
        self.latex_dict_inverse = {} # Initialize inverse dict for quicker reverse lookups
        self.create_dict()

    def create_dict(self):
        # Go through entire label file and populate dictionary
      # print("Passing here")
      Freq_labeled_file = open("./output.txt", "r")

      for line in Freq_labeled_file: #For each line in data file
          token = line.split(',')[1].split(" ")[0]
          latexSymbol = line.split(",")[0]
          # print(r'{' + token + '}')
          self.latex_dict[latexSymbol] = int(token)
          self.latex_dict_inverse[token] = latexSymbol
      Freq_labeled_file.close()

    def map_tokens(self, tex_str_list, batch_size):
        ids_tensor = torch.full((batch_size, self.max_seq_length), self.latex_dict['PAD'], dtype=torch.float32)
        # print('ids_tensor',ids_tensor.shape)
        for row, tex_str in enumerate(tex_str_list):
            tex_str = r'{ ' + tex_str + ' }'
            # print(tex_str)
            tokens = tex_str.split()
            # print('tokens = ',tokens)
            for col, token in enumerate(tokens):
              # print('col = ',col)
              # print('token = ',token)
              if token in self.latex_dict:
                ids_tensor[row, col] = self.latex_dict[token]
              else:
               ids_tensor[row, col] = self.latex_dict['UNK']

        return ids_tensor

    def tokens_to_tex(self, token_vec):
        tex_str = ' '
        for token_id in token_vec.tolist():
            if token_id in self.latex_dict_inverse:
                if self.latex_dict_inverse[token_id] != 'PAD' and self.latex_dict_inverse[token_id] != 'UNK':
                    tex_str += self.latex_dict_inverse[token_id] + ' '

        return tex_str

    def __dict__(self):
        return self.latex_dict

    def __len__(self):
        return len(self.latex_dict)

latex_dict = LatexDictModified()

Creating Database for each character


In [ ]:
import matplotlib.pyplot as plt
from io import BytesIO
from PIL import Image
import numpy as np

def create_latex_image(latex_code, width_px, height_px, dpi=100):
    # Calculate figure size in inches
    figsize = (width_px / dpi, height_px / dpi)

    fig = plt.figure(figsize=figsize)
    plt.text(0.5, 0.5, f'${latex_code}$', fontsize=24, ha='center', va='center')
    plt.axis('off')

    # Save the figure to a BytesIO object
    buffer = BytesIO()
    plt.savefig(buffer, format='png', bbox_inches='tight', pad_inches=0, dpi=dpi)
    # plt.show()
    plt.close(fig)
    buffer.seek(0)

    # Load the image from the buffer, convert to grayscale and to a NumPy array
    image = Image.open(buffer).convert('L')
    return np.array(image)

# Example usage
test_dict = latex_dict.__dict__()
test_keys = list(test_dict.keys())
preferred_width = 128  # in pixels
preferred_height = 32  # in pixels
err_list = []     # List of labels that give error when we input them without anything else like \frac
blank_list = []   # List of labels that generate blank image
image_list = []   # Images generated by the function
label_list = []   # Labels coresponding to the images list
for item in test_keys:
  latex_expression = r' ' + item + ' '
  try:
    latex_image = create_latex_image(latex_expression, preferred_width, preferred_height)
    if (np.all(latex_image == 255)):
      blank_list.append(item)
    else:
      image_list.append(latex_image)
      label_list.append(item)

    # print(item)
  except Exception as error:
    err_list.append(item)
# print(latex_image.shape)
print(len(err_list))
print(err_list)

# print(test_keys[29])
# latex_image = create_latex_image(r' \left( a ' + test_keys[29], preferred_width, preferred_height)


<ipython-input-8-2102a1109320>:10: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`). Consider using `matplotlib.pyplot.close()`.
  fig = plt.figure(figsize=figsize)


34
['{', '}', '\\frac', '\\right)', '\\left(', '\\mathrm', '\\bar', '\\sqrt', '\\\\', '\\mathbf', '\\hat', '\\begin{array}', '\\end{array}', '\\tilde', '\\vec', '\\right]', '\\left[', '\\dot', '\\overline', '\\operatorname', '\\mathcal', '\\left\\{', '\\mathbb', '\\right|', '\\right\\}', '\\left|', '\\Big', '\\displaystyle', '\\widetilde', '\\boldsymbol', '\\operatorname*', '\\big', '\\right.', '\\widehat']


ValueError: ignored

<Figure size 128x32 with 1 Axes>

ValueError: ignored

...

<Figure size 128x32 with 1 Axes>

ValueError: ignored

<Figure size 128x32 with 1 Axes>

ValueError: ignored

<Figure size 128x32 with 1 Axes>

Creating augmented

In [ ]:
from PIL import Image
import numpy as np
import random

def add_random_padding(image, target_height, target_width):
    # Assuming image is a NumPy array
    orig_height, orig_width = image.shape

    # Calculate the total padding required
    padding_height = target_height - orig_height
    padding_width = target_width - orig_width

    # Randomly distribute the padding on each side
    top_padding = random.randint(0, padding_height)
    bottom_padding = padding_height - top_padding
    left_padding = random.randint(0, padding_width)
    right_padding = padding_width - left_padding

    # Create a new white image with the target size
    new_image = np.full((target_height, target_width), 255, dtype=np.uint8)

    # Paste the original image onto the new canvas
    new_image[top_padding:top_padding+orig_height, left_padding:left_padding+orig_width] = image

    return new_image

save_path_img = '/content/char_train'
save_path_labels = '/content/char_labels.txt'

# To remove data in the label file
with open(save_path_labels, 'w') as file:
  file.write('')

# Example usage
# Assuming 'latex_image' is your original NumPy array image

target_height = 64
target_width = 256
# Number of samples we want for each character
Num_samples = 30
for i in range(len(image_list) -2):
  latex_image = image_list[i]
  latex_label = label_list[i]
  for j in range(Num_samples):
    image_number = f"{i*Num_samples+j:07d}"  # Formats the number with leading zeros
    image_name = f"{image_number}.png"
    save_path_pil_img = f"{save_path_img}/{image_name}"
    padded_temp_image = add_random_padding(latex_image, target_height, target_width)
    padded_image_pil = Image.fromarray(padded_temp_image)
    padded_image_pil.save(save_path_pil_img)
    with open(save_path_labels, 'a') as file:
      file.write(f"{latex_label} \n")



# Convert back to PIL image for saving or displaying
#padded_image_pil = Image.fromarray(padded_image)
#padded_image_pil.save('padded_image.png')

CNN Block with 512 channel output

In [8]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader
import torch.optim as optim
import math

class CNN_Block(nn.Module):
    def __init__(self):
        super(CNN_Block, self).__init__()
        # self.conv1 = nn.Conv2d(3, 64, 3)
        self.conv1 = nn.Conv2d(1, 64, 3, padding=1)    # Images are originally one channel, added padding as well
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(64, 128, 3, padding=1)
        # self.pool2 = nn.MaxPool2d(2, 2)
        self.conv3 = nn.Conv2d(128, 256, 3, padding=1)
        self.bn1 = nn.BatchNorm2d(256)
        # self.pool3 = nn.MaxPool2d(2, 2)
        self.conv4 = nn.Conv2d(256, 256, 3, padding=1)
        self.conv5 = nn.Conv2d(256, 256, 3, padding=1)
        self.bn2 = nn.BatchNorm2d(256)
        # self.pool4 = nn.MaxPool2d(2, 2)

        #self.fc1 = nn.Linear(256 * 2 * 8, 1024)  # Adjusted input size based on the output size of the convolutional layers
        #self.fc2 = nn.Linear(1024, 512)


    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = self.pool(self.bn1(F.relu(self.conv3(x))))
        x = self.pool(F.relu(self.conv4(x)))
        x = self.bn2(F.relu(self.conv5(x)))
        #print('x: shape:', x.shape)
        #x = x.view(64,-1)   # Flatten so this can be used in linear layers

        #x = F.relu(self.fc1(x))
        #x = self.fc2(x)

        return x


Defining decoding class: \\
  hidden_size: dimensionality of the hidden layer of the LSTM \\
  batch_size: Size of the batch \\
  seq_length: length of the input sequence \\
  vocab_size: size of our dictionary \\
  emd_size: Number of rows in embedding vector to represent \\
  enc_out: Number of output channels in encoder \\
  

In [9]:
class decoder(nn.Module):
  def __init__(self, hidden_size, batch_size, seq_length, vocab_size, embedding_size, enc_out = 256):
    super(decoder, self).__init__()

    self.hidden_size = hidden_size
    self.batch_size = batch_size
    self.seq_length = seq_length
    self.vocab_size = vocab_size
    self.embedding_size = embedding_size
    self.enc_out = enc_out


    self.cnn_encoder = CNN_Block()
    self.lstm = nn.LSTMCell(hidden_size + embedding_size, hidden_size)
    self.emb = nn.Embedding(vocab_size,embedding_size )

    self.wh_in = nn.Linear(enc_out, hidden_size)
    self.wc_in = nn.Linear(enc_out, hidden_size)
    self.wo_in = nn.Linear(enc_out, hidden_size)

    self.dropout = nn.Dropout(p=0.4)
    self.soft = nn.Softmax(dim = 2)
    self.device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

 # Attention mechanism
    self.beta = nn.Parameter(torch.Tensor(enc_out))
    nn.init.uniform_(self.beta, -1e-2, 1e-2)
    self.W_1 = nn.Linear(enc_out, enc_out, bias=False)
    self.W_2 = nn.Linear(hidden_size, enc_out, bias=False)

    self.W_3 = nn.Linear(hidden_size + enc_out, hidden_size, bias=False)
    self.W_out = nn.Linear(hidden_size, vocab_size, bias=False)


  def forward(self, im, label):
    enc_im = self.encode(im.to(self.device))
    dec_states, o_t = self.init_decoder(enc_im.to(self.device))
    # Suppose we are taking labels with format [Batch, Max Len, Vocab_size]
    max_len = label.shape[1]
    outputs = []
    flag = False
    for i in range(max_len):
      lll = label[:, i, :]
      target_label = torch.argmax(lll.to(self.device), dim = 1, keepdim = True)
      if flag:
        target_label = torch.argmax(outputs[-1].to(self.device), dim = 1, keepdim = True)
        #print("Target label = ", target_label)

      dec_states, output = self.step_decode(dec_states, o_t.to(self.device), enc_im.to(self.device), target_label.to(self.device))
      #print('output',output[1], self.soft(output))
      outputs.append(output.to(self.device))
      flag = True
    outputs = torch.stack(outputs, dim=1)
    outputs = self.soft(outputs)

    return outputs




  def encode(self, im):
    cnn_encode = self.cnn_encoder(im.to(self.device))
    # print("CNN OUTPUT = ", cnn_encode.shape)
    pe = self.PositionalEmbedding2D(self.enc_out, cnn_encode.shape[2], cnn_encode.shape[3]).repeat(cnn_encode.shape[0], 1, 1, 1).to(self.device)
    # print("PE = ", pe.shape)
    # print(pe)
    cnn_encode = cnn_encode + pe
    cnn_encode = cnn_encode.permute(0, 2, 3, 1)
    batch, height, width, channel = cnn_encode.shape
    # pe_enc = self.PositionalEmbedding2D(, )


    cnn_encode = cnn_encode.contiguous().view(batch, height * width, -1)
    return cnn_encode


  def PositionalEmbedding2D(self, D_model,height,width):
    if D_model % 4 != 0:
        raise ValueError("Cannot use sin/cos positional encoding with "
                         "odd dimension (got dim={:d})".format(D_model))
    pe = torch.zeros(D_model,height,width)
    d_model = int(D_model / 2)
    div_term = torch.exp(torch.arange(0., d_model, 2) * -(math.log(10000.0) / d_model))
    pos_w = torch.arange(0., width).unsqueeze(1)
    pos_h = torch.arange(0., height).unsqueeze(1)
    pe[0:d_model:2, :, :] = torch.sin(pos_w * div_term).transpose(0, 1).unsqueeze(1).repeat(1, height, 1)
    pe[1:d_model:2, :, :] = torch.cos(pos_w * div_term).transpose(0, 1).unsqueeze(1).repeat(1, height, 1)
    pe[d_model::2, :, :] = torch.sin(pos_h * div_term).transpose(0, 1).unsqueeze(2).repeat(1, 1, width)
    pe[d_model + 1::2, :, :] = torch.cos(pos_h * div_term).transpose(0, 1).unsqueeze(2).repeat(1, 1, width)

    return pe.permute(0, 1, 2)


  def init_decoder(self, enc_im):
    enc_im_mean = enc_im.mean(dim=1)
    h = torch.tanh(self.wh_in(enc_im_mean))
    c = torch.tanh(self.wc_in(enc_im_mean))
    o_init = torch.tanh(self.wo_in(enc_im_mean))
    return (h, c), o_init



  def step_decode(self, dec_states, o_t, enc_im, target_label ):
    y_emb = self.emb(target_label).squeeze(1)
    # print("y_emb = ", y_emb.shape)
    # print("o_T = ", o_t.shape)
    inp = torch.cat([y_emb, o_t], dim = 1)
    # print("inp = ", inp.shape)
    # print("h = ", dec_states[0].shape)
    h, c = self.lstm(inp, dec_states)
    h = self.dropout(h)
    c = self.dropout(c)

    contex = self.attentionMechanism(enc_im, h)

    o_t = self.W_3(torch.cat([h, contex], dim=1)).tanh()
    o_t = self.dropout(o_t)
    output = self.W_out(o_t)

    return (h, c), output


  def attentionMechanism(self, enc_im, h):
    alpha = torch.tanh(self.W_1(enc_im) + self.W_2(h).unsqueeze(1))
    alpha = torch.sum(self.beta * alpha, dim=-1)  # [B, L]
    alpha = F.softmax(alpha, dim=-1)  # [B, L]

    # cal context: [B, C]
    contex = torch.bmm(alpha.unsqueeze(1), enc_im)
    contex = contex.squeeze(1)
    return contex



In [10]:
a = [1,2,3,4]
a[1:2]

[2]

Initializing the database and code output

In [11]:
'''
Initialize dataset and image preprocessing

NOTE:
    Some of the images in the dataset are corrupted. To deal with this,
    there is a check for each image to ensure that it can be loaded.
'''
import torchvision.transforms as transforms

reduced_imsize = (32, 128)  # Images are reduced to this size

# Define the transform pipeline - add normalization?
transform = transforms.Compose([
    transforms.Resize(reduced_imsize),
    transforms.ToTensor(),
])

train_root = '/content/char_train/'
seq_length = 4        # Maximum number of sequence length i

train_dataset = LatexDataset(transform=transform, dataroot=train_root, max_seq_length=seq_length)
test_dataset = LatexDataset(transform=transform, dataroot=test_root)
val_dataset = LatexDataset(transform=transform, dataroot=val_root)

# Device settings
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print(f'Device: {device}')

Device: cuda:0


In [19]:
'''
Initialize hyperperameters, trainloader, and dictionary of LaTeX token mappings
'''

# Hyperparameters
batch_size = 10
learning_rate = 0.0001
weight_decay = 0 # (L2 penalty)

latex_dict = LatexDictModified(max_seq_length=seq_length)

trainloader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=8, drop_last=True)

# print(f'Dictionary length: {latex_dict.__len__()}')
# print(f'Dictionary: {latex_dict.__dict__()}')

In [21]:
'''
Initialize Model

Initialize Loss Functions:
1. Normal Cross-Entropy Loss between prediction and label
2. LaTeX compile test:
    - Custom function, returns True if code can compile into LaTeX, False if not

Initialize Optimizer:
1. Adam Optimizer
'''

    # Hyperparameters
# embedding_size = 80; # number of rows in the E-matrix
# o_layer_size = 700  ;  # size of o-vektorn TODO: What should this be?
# hidden_size = 700  ;
# sequence_length = 700  ; vocab_size = 700  ;
# batch_size = 16
# n_epochs = 2
# beam_size = 5

hidden_size = 100
batch_size = 10
# seq_length = 300
vocab_size = latex_dict.__len__()
embedding_size = 80
model = decoder(hidden_size, batch_size, seq_length, vocab_size, embedding_size).to(device)

# model = Model(embedding_size=embedding_size, hidden_size=hidden_size, batch_size=batch_size, sequence_length=sequence_length, vocab_size=vocab_size, o_layer_size = o_layer_size).to(device)
criterion = nn.CrossEntropyLoss()   ## May need to change this
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate, weight_decay=weight_decay)

In [22]:
print(vocab_size)

155


Testing the dimensionality of CNN output for LSTM inputs

In [23]:
pbar_test = trainloader
images, y = next(iter(pbar_test))
cnn_test_block = CNN_Block()
cnn_output = cnn_test_block(images)
print(cnn_output.shape)

torch.Size([10, 256, 2, 8])


In [24]:
from tqdm import tqdm

'''
Training Loop
'''




num_epoch = 200
model.train()
counter = 0
for epoch in range(num_epoch):
    print('epoch:', epoch)
    pbar = tqdm(trainloader)
    for images, y in pbar:

        images = images.to(device)              # Send to gpu
        # print("y = ",y)
        y_vec = latex_dict.map_tokens(list(y), batch_size=batch_size)
        # print(" ")
        # print("y_vec shape = ", y_vec.shape)
        #print(y_vec)
        # latexx = latex_dict.tokens_to_tex(y_vec[0])
        #print(latexx)
        # print(y_vec[0,:])
        # print(torch.max(y_vec[0,:]))

        y_vec = torch.tensor(y_vec, dtype=torch.long)
        y_vec = torch.LongTensor(y_vec)
        # print(y_vec[9])
        one_hot = torch.zeros(y_vec.shape[0], y_vec.shape[1], vocab_size)
        for i in range(y_vec.shape[0]):
            for j in range(y_vec.shape[1]):
                one_hot[i,j,y_vec[i,j]] = 1
        # one_hot = torch.nn.functional.one_hot(y_vec, num_classes=vocab_size)
        # print(one_hot[0,0,:])
        #print(one_hot
        one_hot = one_hot.to(device)                # Send to gpu
        #print(images.shape)
        #V = images.reshape(images.shape[0], images.shape[1] * images.shape[2], images.shape[3])
        predictions = model.forward(images,one_hot)        # Get predictions

        # print(torch.argmax(predictions[1]),torch.argmax(one_hot[1]))
        # print(predictions.shape,one_hot.shape)
        one_hot = torch.tensor(one_hot, dtype=torch.float)

        # print("Prediction = ", predictions[0,1:3,:])
        # print("sum = ", torch.sum(predictions, 2))
        #print('y_vec',y_vec.shape)
        # print(" ")
        # print('Prediction = ',predictions.shape)
        # print('one_hot = ',one_hot.shape)
        counter += 1
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1)
        loss1 = criterion(predictions, one_hot)   # Calculate first loss function
        if (counter % 50 == 0):
            print('y_real',y_vec[9])
            print('y_pred',torch.argmax(predictions, dim=2)[9])
            print('loss',loss1)
            counter = 1
        # loss2 = can_compile(precitions)         # Check if output can compile

        optimizer.zero_grad()

        loss1.backward()
        optimizer.step()


epoch: 0


  0%|          | 0/237 [00:00<?, ?it/s]<ipython-input-24-5f5af669ffc1>:29: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  y_vec = torch.tensor(y_vec, dtype=torch.long)
<ipython-input-24-5f5af669ffc1>:46: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  one_hot = torch.tensor(one_hot, dtype=torch.float)
 25%|██▌       | 60/237 [00:01<00:03, 53.13it/s]

y_real tensor([154,  79, 154, 153])
y_pred tensor([ 73, 153, 153, 153], device='cuda:0')
loss tensor(0.0357, device='cuda:0', grad_fn=<DivBackward1>)


 46%|████▌     | 108/237 [00:02<00:02, 56.78it/s]

y_real tensor([154, 135, 154, 153])
y_pred tensor([154, 154, 153, 153], device='cuda:0')
loss tensor(0.0344, device='cuda:0', grad_fn=<DivBackward1>)


...

 76%|███████▌  | 179/237 [00:03<00:01, 50.90it/s]

y_real tensor([154, 135, 154, 153])
y_pred tensor([154, 135, 154, 153], device='cuda:0')
loss tensor(0.0226, device='cuda:0', grad_fn=<DivBackward1>)


 96%|█████████▌| 228/237 [00:04<00:00, 53.29it/s]

y_real tensor([154, 122, 154, 153])
y_pred tensor([154, 122, 154, 153], device='cuda:0')
loss tensor(0.0230, device='cuda:0', grad_fn=<DivBackward1>)


100%|██████████| 237/237 [00:05<00:00, 46.49it/s]


In [26]:
print(predictions.shape)
y = torch.argmax(predictions, dim=2)
print()
print(y[9],y_vec[9])
print(y[8],y_vec[8])
print(y[7],y_vec[7])
print(y[6],y_vec[6])
print(y[5],y_vec[5])
print(y[4],y_vec[4])
print(y[3],y_vec[3])
print(y[2],y_vec[2])
print(y[1],y_vec[1])
print(y[0],y_vec[0])
print(latex_dict.tokens_to_tex(y_vec[1]))

torch.Size([10, 4, 155])

tensor([154,  32, 154, 153], device='cuda:0') tensor([154,  32, 154, 153])
tensor([154,  18, 154, 153], device='cuda:0') tensor([154,  18, 154, 153])
tensor([154,  55, 154, 153], device='cuda:0') tensor([154,  55, 154, 153])
tensor([154, 113, 154, 153], device='cuda:0') tensor([154,  70, 154, 153])
tensor([154,  62, 154, 153], device='cuda:0') tensor([154,  62, 154, 153])
tensor([154,   2, 154, 153], device='cuda:0') tensor([154,   2, 154, 153])
tensor([154,  36, 154, 153], device='cuda:0') tensor([154,  36, 154, 153])
tensor([154,  53, 154, 153], device='cuda:0') tensor([154,  53, 154, 153])
tensor([154,  19, 154, 153], device='cuda:0') tensor([154,  19, 154, 153])
tensor([154,  71, 154, 153], device='cuda:0') tensor([154,  71, 154, 153])
 


In [29]:
model_path = '/content/drive/MyDrive/Deep Learning Proj/bestmodel.pth'

# Save the entire model (including architecture and learned parameters)
torch.save(model.state_dict(), model_path)

In [47]:
model.eval()  # Set the model to evaluation mode
total_test_loss = 0.0
total_correct_predictions = 0
total_samples = 0

with torch.no_grad():  # No need to compute gradients during testing
    for images, y in tqdm(trainloader):  # Assuming you have a separate testloader
        images = images.to(device)
        y_vec = latex_dict.map_tokens(list(y), batch_size=batch_size)
        y_vec = torch.tensor(y_vec, dtype=torch.long).to(device)
        y_vec = torch.LongTensor(y_vec.tolist())
        one_hot = torch.zeros(y_vec.shape[0], y_vec.shape[1], vocab_size)

        # ... (similar one-hot encoding as in the training loop)

        one_hot = one_hot.to(device)

        predictions = model.forward(images, one_hot)
        one_hot = torch.tensor(one_hot, dtype=torch.float)

        loss = criterion(predictions, one_hot)
        total_test_loss += loss.item()

        # Calculate accuracy
        predicted_labels = torch.argmax(predictions, dim=2)
        predicted_labels.to(device)
        y_vec = y_vec.to(device)
        correct_predictions = torch.sum(predicted_labels == y_vec)/seq_length
        total_correct_predictions += correct_predictions.item()

        total_samples += images.size(0)

# Calculate average test loss and accuracy
average_test_loss = total_test_loss / len(trainloader)
accuracy = total_correct_predictions / total_samples

print(f"Accuracy: {accuracy}")
print(predicted_labels)
print(y_vec)

  0%|          | 0/237 [00:00<?, ?it/s]<ipython-input-47-f6ce5dce51c0>:10: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  y_vec = torch.tensor(y_vec, dtype=torch.long).to(device)
<ipython-input-47-f6ce5dce51c0>:19: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  one_hot = torch.tensor(one_hot, dtype=torch.float)
100%|██████████| 237/237 [00:07<00:00, 32.08it/s] 

Accuracy: 0.990506329113924
tensor([[154,  62, 154, 153],
        [154,  83, 154, 153],
        [154,  24, 154, 153],
        [154,   1, 154, 153],
        [154,  91, 154, 153],
        [154,  55, 154, 153],
        [154,  93, 154, 153],
        [154,  35, 154, 153],
        [154,   9, 154, 153],
        [154, 135, 154, 153]], device='cuda:0')
tensor([[154,  62, 154, 153],
        [154,  83, 154, 153],
        [154,  24, 154, 153],
        [154,   1, 154, 153],
        [154,  91, 154, 153],
        [154,  55, 154, 153],
        [154,  93, 154, 153],
        [154,  35, 154, 153],
        [154,   9, 154, 153],
        [154, 135, 154, 153]], device='cuda:0')
